In [1]:
from data_utils import *
from models import *
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
data_dir = "data/"
dataObject = xrdData("data/", device)

In [3]:
dataObject.make_datasets(0.01, composition_embedding="composition1D")
# Create DataLoaders for train and validation sets
train_loader = DataLoader(dataObject.torch_datasets['train'], batch_size=256, shuffle=True)
valid_loader = DataLoader(dataObject.torch_datasets['val'], batch_size=256, shuffle=False)  

In [4]:
# Create the model instance and move it to the selected device
output_dim = 230  # Output dimension
model = XRD_C_SymNet(in_channels=1, output_dim=output_dim, composition_model= None).to(device)

# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [5]:
max_epochs = 100
metrics = ["accuracy", "loss"]

log = {
    f"{type}": {f"{metric}" : np.zeros(max_epochs) for metric in metrics} for type in ['train', 'val']     
}

for epoch in range(max_epochs):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for xrd, composition, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(xrd, composition)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_valid_loss = 0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():  # No gradients needed for validation
        for xrd, composition, targets in valid_loader:
            outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_valid += targets.size(0)
            correct_valid += (predicted == targets).sum().item()

    valid_accuracy = 100 * correct_valid / total_valid

    total_train_loss = total_train_loss / len(train_loader)
    validation_loss = total_valid_loss / len(valid_loader)

    print(f"Epoch {epoch+1}, Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}%, Validation Loss: {validation_loss}, Validation Accuracy: {valid_accuracy}%")

    log['train']['accuracy'][epoch] = (train_accuracy)
    log['train']['loss'][epoch] =(total_train_loss)

    log['val']['accuracy'][epoch] =(valid_accuracy)
    log['val']['loss'][epoch] =(validation_loss)

Epoch 1, Training Loss: 5.402815818786621, Training Accuracy: 4.722222222222222%, Validation Loss: 5.409026357862684, Validation Accuracy: 14.746849436214902%
Epoch 2, Training Loss: 5.053048372268677, Training Accuracy: 15.0%, Validation Loss: 5.317891253365411, Validation Accuracy: 14.746849436214902%
Epoch 3, Training Loss: 4.634365558624268, Training Accuracy: 14.722222222222221%, Validation Loss: 5.088983429802789, Validation Accuracy: 14.746849436214902%
Epoch 4, Training Loss: 4.333333492279053, Training Accuracy: 14.722222222222221%, Validation Loss: 4.792791128158569, Validation Accuracy: 14.746849436214902%
Epoch 5, Training Loss: 3.986951231956482, Training Accuracy: 14.444444444444445%, Validation Loss: 4.6260154247283936, Validation Accuracy: 14.746849436214902%
Epoch 6, Training Loss: 3.7074244022369385, Training Accuracy: 14.166666666666666%, Validation Loss: 4.468586020999485, Validation Accuracy: 15.189033827105904%


KeyboardInterrupt: 

In [22]:
model_dir = "models/"

date = datetime.now()
model_name = f"first_model_{date.month}_{date.day}"

os.makedirs(os.path.join(model_dir, model_name), exist_ok=True)

torch.save(model, os.path.join(model_dir, model_name, 'model.pth'))

for data_type, metrics_dict in log.items():
    for metric, array in metrics_dict.items():
        filename = f"{data_type}_{metric}.npy"  # Construct filename, e.g., "train_accuracy.npy"
        filename = os.path.join(model_dir, model_name, filename)
        np.save(filename, array)  # Save the array to a file